In [1]:
import pandas as pd

In [2]:
# Create the catalog of which RIDS participated in which phase
data = pd.read_csv("Datasets/Raw Data Files/ROSTER - Roster [ADNI1,GO,2,3,4].csv")
data = data[["PHASE", "RID"]].sort_values("RID")
print(data)
data.to_csv("Datasets/RIDS_Phase_Catalog.csv")

       PHASE    RID
0      ADNI1      1
1      ADNI1      2
4603   ADNI3      2
2399   ADNI2      2
1585  ADNIGO      2
...      ...    ...
6487   ADNI4  10133
6488   ADNI4  10134
6489   ADNI4  10135
6490   ADNI4  10136
6491   ADNI4  10137

[6492 rows x 2 columns]


In [ ]:
current_labels_catalog = pd.read_csv("Dallan Work/visit_dx_combined_updated_old.csv")
current_labels_catalog.drop(["Unnamed: 1", "RID.1"], axis=1, inplace=True)

# Add stuff from ADNI merge
adni_merge = pd.read_csv("Datasets/Raw Data Files/ADNIMERGE - Key ADNI tables merged into one table [ADNI1,GO,2,3].csv")
adni_merge_interest = adni_merge[["RID", "DX", "ADAS13", "ADAS13_bl", "M"]]
adni_merge_interest = adni_merge_interest.rename(columns={"M": "VISMONTH"})
adni_merge_interest["DX"] = adni_merge_interest["DX"].map({"CN": 1, "MCI": 2, "Dementia": 3})
new_labels_catalog = current_labels_catalog.merge(adni_merge_interest, on=["RID", "VISMONTH"], how="left")

# a VERY temporary solution here go back and clean this up sometime
new_labels_catalog["DIAGNOSIS2"] = new_labels_catalog["DIAGNOSIS"]
new_labels_catalog["DX2"] = new_labels_catalog["DX"]

new_labels_catalog.to_csv("Dallan Work/visit_dx_combined_updated.csv", index=False)

/tmp/ipykernel_61427/775417084.py:5: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  adni_merge = pd.read_csv("Datasets/Raw Data Files/ADNIMERGE - Key ADNI tables merged into one table [ADNI1,GO,2,3].csv")


In [ ]:
# If you want to check on if two of your columns are actually the same

def column_correlation_and_mismatches(df, col1, col2):
    # Drop NaNs for correlation calculation
    clean_df = df[[col1, col2]].dropna()
    correlation = clean_df.corr().iloc[0, 1]
    
    # Find mismatched pairs
    mismatches = clean_df[clean_df[col1] != clean_df[col2]]

    return correlation, mismatches

# Example usage
data = {
    'A': [1, 2, 3, 4, None, 6, 7],
    'B': [1, 4, None, 4, 10, 12, 7]
}
df = pd.DataFrame(data)

correlation, mismatches = column_correlation_and_mismatches(new_labels_catalog, 'TOTAL13', 'ADAS13')
print(f'Correlation: {correlation}')
print('Mismatched pairs:')
print(mismatches)


Correlation: 0.9980181360333971
Mismatched pairs:
         TOTAL13  ADAS13
202     8.000000   13.00
308    21.000000   20.00
332     7.750000    3.00
774     6.665000    7.00
962     7.886667    5.00
...          ...     ...
17720  21.330000   23.33
17734  27.330000   28.33
17755   5.000000   15.00
17804   5.670000   16.67
17806  11.330000   21.67

[224 rows x 2 columns]
